In [1]:
#Creates a class for chalets that takes in serial number of chalet, its cost and its capacity
class Chalets:
    def __init__(self):
        self.cost = 0
        self.capacity = 0
        self.serial = 0
        self.assigned_students = []    #This is the list of all students that are assigned to a particular chalet
        self.cant_be_assigned_students = []*100  #This is the list of all students that can't be assigned   
        

    def addChalet(self, serial, cost, capacity):
        self.cost = cost
        self.capacity = capacity
        self.serial = serial
        self.assigned_students = []*capacity    #This is the list of all students that are assigned to a particular chalet
        self.cant_be_assigned_students = []*100  #This is the list of all students that can't be assigned   
        
    def addStudent(self, student):
        if student.serial_number in self.cant_be_assigned_students: #This function checks whether a student is in existing list of cant be assigned students
            pass
        else:
            if len(self.assigned_students) < self.capacity and student.isAssigned == False: #Checks whether number of already assigned students are less than capacity 
                if len(set(student.conflicts).intersection(set(self.assigned_students))) == 0 :
                    self.assigned_students.append(student.serial_number)
                    self.cant_be_assigned_students.extend(student.conflicts)
                    student.isAssignedToChalet(True)
            else:
                pass

    def removeStudent(self, student):
        if student.serial_number in self.assigned_students:
            self.assigned_students.remove(student.serial_number)
            self.capacity -= 1
            for i in range(len(student.conflicts)):
                self.cant_be_assigned_students.remove(student.conflicts[i])
            student.isAssigned = False
                
    def __str__(self):
        return "Chalet with {} serial number has {} students in it. They are {}".format(self.serial, len(self.assigned_students), self.assigned_students)


class Student:
    def __init__(self):
        self.serial_number = 0
        self.conflicts = []  #List of all students' serial number having conflict with given student
        self.number_conflicts = 0
        self.isAssigned = False

    def addStudent(self, serial_number, number_conflicts, list_of_conflicts):
        self.serial_number = serial_number
        self.conflicts = []*number_conflicts  #List of all students' serial number having conflict with given student
        self.number_conflicts = number_conflicts
        self.conflicts.extend(list_of_conflicts)

    def isAssignedToChalet(self, Boolean):
        self.isAssigned = Boolean #True if assigned else false 

    def __str__(self):
        return "Student has {} serial number. She is in conflict with {} students. Their serial numbers are {}".format(self.serial_number, self.number_conflicts, self.conflicts)
        




    

In [2]:
####Genetic Algorithm############

import random
#Choosing Student with maximum conflicts by sorting students list as per number of conflicts 
#Students_list.sort(key=lambda student: student.number_conflicts, reverse=True)

# Sort chalets by the number of available places in descending order
#Chalets_list.sort(key = lambda chalet: chalet.capacity - len(chalet.assigned_students), reverse = True)

#Creating population 
pop = [[] for _ in range(10)] 
n = 0
while n < 10:
    
    #Reads a given file and returns Chalets Lists and Students Lists. Finds number of chalets and number of students 
    f  = open('instance12_12.txt', 'r')
    M, N = map(int, f.readline().split())
    
    #Creates list of chalets object. Each object is of class chalet 
    Chalets_list = []
    for i in range(1, M+1):
        cost, capacity = map(int, f.readline().split())
        chalet = Chalets()
        chalet.addChalet(i, cost, capacity)
        Chalets_list.append(chalet)
    
    #Creates list of student objects. Each object is of class student 
    Students_list = []
    for i in range(0 , N):
        a = f.readline().split()
        number_conflicts = int(a[0])
        list_of_conflicts = []*number_conflicts
        for j in range(1, number_conflicts+1):
            list_of_conflicts.append(int(a[j]))
        student = Student()
        student.addStudent(i, number_conflicts, list_of_conflicts)
        Students_list.append(student)
    
    #introducing randomization
    #Storing first three elements of Student list in another list
    #first_three_Students_list = Students_list[:3]
    
    #Shuffling first three elements which are present in the new list
    random.shuffle(Students_list)
    
    #Combining the shuffled elements with the rest of the elements of original list
    new_student_list = Students_list
    
    #Storing first three elements of Chalets list in another list
    #first_three_Chalets_list = Chalets_list[:3]
    
    #Shuffling first three elements which are present in the new list
    random.shuffle(Chalets_list)
    
    #Combining the shuffled elements with the rest of the elements of original list
    new_chalets_list = Chalets_list
      
    for student in new_student_list:
        if not student.isAssigned:  # Check if the student is not already assigned   
        
        # First priority: Choose from rented chalets with highest available places
        #Finding chalet with maximum available space and is rented 
            rented_chalet_with_high_place = Chalets()
            for chalet in new_chalets_list:
                if len(chalet.assigned_students) > 0: #ensures that chalet is rented
                    if (chalet.capacity - len(chalet.assigned_students)) > (rented_chalet_with_high_place.capacity - len(rented_chalet_with_high_place.assigned_students)):
                        rented_chalet_with_high_place = chalet
            rented_chalet_with_high_place.addStudent(student)
        # Exit the loop once assigned to a chalet
                
        # Second priority: Choose from unrented chalets based on capacity/cost score
        if not student.isAssigned:
            max_score = 0
            selected_chalet = None
            for chalet in new_chalets_list:
                if len(chalet.assigned_students) == 0:  # Check if chalet is unrented
                    score = chalet.capacity / chalet.cost
                    if score > max_score:
                        max_score = score
                        selected_chalet = chalet
                    
            if selected_chalet is not None:
                selected_chalet.addStudent(student)
                student.isAssigned = True
            
    # Attempt to optimize by exchanging students between cottages
    #cheapest_k = None
    for j in range(0,len(new_chalets_list)):
        if len(new_chalets_list[j].assigned_students) > 0:  # Only consider rented cottages
            num_students_in_j = len(new_chalets_list[j].assigned_students)
            
            for k in range(0, len(new_chalets_list)):
                if len(new_chalets_list[k].assigned_students) == 0:  # Only consider non-rented cottages
                    if new_chalets_list[k].capacity >= num_students_in_j and new_chalets_list[k].cost < new_chalets_list[j].cost:
                        #print("Found a cheaper cottage for relocation. Its serial number is {}".format(k+1))
                        #print(cheapest_k)
                    
                        #if cheapest_k is not None:
                # Reallocate students from j to cheapest_k
                        for student_serial in new_chalets_list[j].assigned_students[:]:
                            for student in new_student_list: 
                                if student.serial_number == student_serial:
                                    #print("Student with {} serial number will be relocated".format(student_serial))
                                    new_chalets_list[j].removeStudent(student)        
                                    #print(Chalets_list[j])
                                    #print("Student will be removed from {} cottage".format(j+1))
                                    new_chalets_list[k].addStudent(student)
                                    #print(Chalets_list[k])
                                    #print("Student will be removed from {} cottage".format(j+1))
    
    #Creating chromosomes 
    new_student_list.sort(key = lambda student : student.serial_number)
    
    for student in new_student_list:
        for chalet in new_chalets_list:
            if student.serial_number in chalet.assigned_students:
                pop[n].append(chalet.serial)
    n += 1

In [8]:
#Evaluating fitness of chromosomes 
chromosome_cost = []
for row in pop:
    cost = 0
    for serial in set(row):
        for chalet in new_chalets_list:
            if chalet.serial == serial:
               cost += chalet.cost
    chromosome_cost.append(cost)



In [4]:
pop

[[12, 9, 9, 1, 9, 11, 1, 1, 1, 11, 4, 12],
 [11, 5, 11, 4, 12, 4, 4, 12, 12, 9, 4, 5],
 [7, 9, 4, 11, 11, 4, 11, 11, 12, 12, 4, 9],
 [9, 7, 9, 11, 7, 12, 11, 11, 11, 5, 9, 12],
 [9, 5, 9, 11, 5, 12, 6, 6, 9, 4, 4, 5],
 [5, 7, 7, 9, 7, 12, 5, 4, 11, 11, 5, 7],
 [9, 11, 9, 7, 11, 4, 7, 7, 11, 9, 12, 12],
 [9, 6, 9, 11, 6, 6, 11, 6, 11, 4, 12, 9],
 [2, 9, 11, 5, 4, 6, 4, 12, 2, 11, 11, 6],
 [6, 1, 11, 6, 1, 4, 4, 11, 6, 12, 9, 4]]

In [9]:
chromosome_cost

[144, 138, 144, 161, 178, 186, 144, 136, 242, 184]

In [ ]:
#Selecting four chromosomes with least cost
